<a href="https://colab.research.google.com/github/Nesta-gitU/EMOTE/blob/master/Practical_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

------
**You cannot save any changes you make to this file, so please make sure to save it on your Google Colab drive or download it as a .ipynb file.**

------



Practical 1: Sentiment Detection in Movie Reviews
========================================



This practical concerns detecting sentiment in movie reviews. This is a typical NLP classification task.
In [this file](https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json) (80MB) you will find 1000 positive and 1000 negative **movie reviews**.
Each review is a **document** and consists of one or more sentences.

To prepare yourself for this practical, you should
have a look at a few of these texts to understand the difficulties of
the task: how might one go about classifying the texts? You will write
code that decides whether a movie review conveys positive or
negative sentiment.

Please make sure you have read the following paper:

>   Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan
(2002).
[Thumbs up? Sentiment Classification using Machine Learning
Techniques](https://dl.acm.org/citation.cfm?id=1118704). EMNLP.

Bo Pang et al. introduced the movie review sentiment
classification task, and the above paper was one of the first papers on
the topic. The first version of your sentiment classifier will do
something similar to Pang et al.'s system. If you have questions about it,
you should resolve you doubts as soon as possible with your TA.


**Advice**

Please read through the entire practical and familiarise
yourself with all requirements before you start coding or otherwise
solving the tasks. Writing clean and concise code can make the difference
between solving the assignment in a matter of hours, and taking days to
run all experiments.


**Implementation**

While we inserted code cells to indicate where you should implement your own code, please feel free to add/remove code blocks where you see fit (but make sure that the general structure of the assignment is preserved). Also, please keep in mind that it is always good practice to structure your code properly, e.g., by implementing separate classes and functions that can be reused.

## Environment

All code should be written in **Python 3**.
This is the default in Google Colab.

In [ ]:
!python --version

Python 3.10.12


If you want to run code on your own computer, then download this notebook through `File -> Download .ipynb`.
The easiest way to
install Python is through downloading
[Anaconda](https://www.anaconda.com/download).
After installation, you can start the notebook by typing `jupyter notebook filename.ipynb`.
You can also use an IDE
such as [PyCharm](https://www.jetbrains.com/pycharm/download/) to make
coding and debugging easier. It is good practice to create a [virtual
environment](https://docs.python.org/3/tutorial/venv.html) for this
project, so that any Python packages don’t interfere with other
projects.


**Learning Python 3**

If you are new to Python 3, you may want to check out a few of these resources:
- https://learnxinyminutes.com/docs/python3/
- https://www.learnpython.org/
- https://docs.python.org/3/tutorial/

In [22]:
import math
import os
import sys
from subprocess import call
from nltk import FreqDist
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
import sklearn as sk
from google.colab import drive
import pickle
import json
from collections import Counter
import requests
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from copy import deepcopy
import sklearn
from sklearn.svm import LinearSVC

#check version of LinearSVC
print("scikit-learn version:", sklearn.__version__)
#!pip install sklearn==1.3.2

scikit-learn version: 1.2.2
ERROR: Could not find a version that satisfies the requirement sklearn==1.3.2 (from versions: 0.0, 0.0.post1, 0.0.post2, 0.0.post4, 0.0.post5, 0.0.post7, 0.0.post9, 0.0.post10, 0.0.post11)
ERROR: No matching distribution found for sklearn==1.3.2


## Loading the data

**Download the sentiment lexicon and the movie reviews dataset.**

In [2]:
# download sentiment lexicon
!wget https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
# download review data
!wget https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json

--2023-11-07 19:05:22--  https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662577 (647K) [text/plain]
Saving to: ‘sent_lexicon’

sent_lexicon        100%[===================>] 647.05K  3.89MB/s    in 0.2s    

2023-11-07 19:05:22 (3.89 MB/s) - ‘sent_lexicon’ saved [662577/662577]

--2023-11-07 19:05:23--  https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.

**Load the movie reviews.**

Each word in a review comes with its part-of-speech tag. For documentation on POS-tags, see https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf.


In [46]:
# file structure:
# [
#  {"cv": integer, "sentiment": str, "content": list}
#  {"cv": integer, "sentiment": str, "content": list}
#   ..
# ]
# where `content` is a list of sentences,
# with a sentence being a list of (token, pos_tag) pairs.


with open("reviews.json", mode="r", encoding="utf-8") as f:
  reviews = json.load(f)

print("Total number of reviews:", len(reviews), '\n')

def print_sentence_with_pos(s):
  print(" ".join("%s/%s" % (token, pos_tag) for token, pos_tag in s))

for i, r in enumerate(reviews):
  print(r["cv"], r["sentiment"], len(r["content"]))  # cv, sentiment, num sents
  print_sentence_with_pos(r["content"][0])
  if i == 4:
    break

c = Counter()
pos_tags = []
for review in reviews:
  for sentence in review["content"]:
    for token, pos_tag in sentence:
      c[token.lower()] += 1
      pos_tags.append(pos_tag)

print("\nNumber of word types:", len(c))
print("Number of word tokens:", sum(c.values()))

print(list(set(pos_tags)))
print("\nMost common tokens:")
for token, count in c.most_common(20):
  print("%10s : %8d" % (token, count))


Total number of reviews: 2000 

0 NEG 29
Two/CD teen/JJ couples/NNS go/VBP to/TO a/DT church/NN party/NN ,/, drink/NN and/CC then/RB drive/NN ./.
1 NEG 11
Damn/JJ that/IN Y2K/CD bug/NN ./.
2 NEG 24
It/PRP is/VBZ movies/NNS like/IN these/DT that/WDT make/VBP a/DT jaded/JJ movie/NN viewer/NN thankful/JJ for/IN the/DT invention/NN of/IN the/DT Timex/NNP IndiGlo/NNP watch/NN ./.
3 NEG 19
QUEST/NN FOR/IN CAMELOT/NNP ``/`` Quest/NNP for/IN Camelot/NNP ''/'' is/VBZ Warner/NNP Bros./NNP '/POS first/JJ feature-length/JJ ,/, fully-animated/JJ attempt/NN to/TO steal/VB clout/NN from/IN Disney/NNP 's/POS cartoon/NN empire/NN ,/, but/CC the/DT mouse/NN has/VBZ no/DT reason/NN to/TO be/VB worried/VBN ./.
4 NEG 38
Synopsis/NNPS :/: A/DT mentally/RB unstable/JJ man/NN undergoing/VBG psychotherapy/NN saves/VBZ a/DT boy/NN from/IN a/DT potentially/RB fatal/JJ accident/NN and/CC then/RB falls/VBZ in/IN love/NN with/IN the/DT boy/NN 's/POS mother/NN ,/, a/DT fledgling/NN restauranteur/NN ./.

Number of wo

#(1) Lexicon-based approach (3.5pts)



A traditional approach to classify documents according to their sentiment is the lexicon-based approach. To implement this approach, you need a **sentiment lexicon**, i.e., a list of words annotated with a sentiment label (e.g., positive and negative, or a score from 0 to 5).

In this practical, you will use the sentiment
lexicon released by Wilson et al. (2005).

> Theresa Wilson, Janyce Wiebe, and Paul Hoffmann
(2005). [Recognizing Contextual Polarity in Phrase-Level Sentiment
Analysis](http://www.aclweb.org/anthology/H/H05/H05-1044.pdf). HLT-EMNLP.

Pay attention to all the information available in the sentiment lexicon. The field *word1* contains the lemma, *priorpolarity* contains the sentiment label (positive, negative, both, or neutral), *type* gives you the magnitude of the word's sentiment (strong or weak), and *pos1* gives you the part-of-speech tag of the lemma. Some lemmas can have multiple part-of-speech tags and thus multiple entries in the lexicon. The path of the lexicon file is `"sent_lexicon"`.


In [4]:
with open("sent_lexicon", mode="r", encoding="utf-8") as f:
  line_cnt = 0
  for line in f:
    print(line.strip())
    line_cnt += 1
    if line_cnt > 4:
      break

type=weaksubj len=1 word1=abandoned pos1=adj stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandonment pos1=noun stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandon pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abase pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abasement pos1=anypos stemmed1=y priorpolarity=negative


Lexica such as this can be used to solve
the classification task without using Machine Learning. For example, one might look up every word $w_1 ... w_n$ in a document, and compute a **binary score**
$S_{binary}$ by counting how many words have a positive or a
negative label in the sentiment lexicon $SLex$.

$$S_{binary}(w_1 w_2 ... w_n) = \sum_{i = 1}^{n}\text{sign}(SLex\big[w_i\big])$$

where $\text{sign}(SLex\big[w_i\big])$ refers to the polarity of $w_i$.

**Threshold.** On average, there are more positive than negative words per review (~7.13 more positive than negative per review) to take this bias into account you should use a threshold of **8** (roughly the bias itself) to make it harder to classify as positive.

$$
\text{classify}(S_{binary}(w_1 w_2 ... w_n)) = \bigg\{\begin{array}{ll}
        \text{positive} & \text{if } S_{binary}(w_1w_2...w_n) > threshold\\
        \text{negative} & \text{otherwise}
        \end{array}
$$


#### (Q1.1) Implement this approach and report its classification accuracy. (1 pt)

In [ ]:
def read_lexicon(path='sent_lexicon'):
    """ Reads the lexicon form a file into a dictionary """
    lexicon = {} #defaultdict(dict)
    list_to_delete = []
    with open("sent_lexicon", mode="r", encoding="utf-8") as file:
        for line in file:
            type_, len_, word, pos1, stemmed1, priorpolarity = line.strip().split()
            if word.split('=')[-1] in lexicon:
                if lexicon[word.split('=')[-1]]['priorpolarity'] != priorpolarity.split('=')[-1]:
                    list_to_delete.append(word.split('=')[-1])
#                     continue

            lexicon[word.split('=')[-1]] = {
                'type': type_.split('=')[-1],
                'len': len_.split('=')[-1],
                'stemmed1': stemmed1.split('=')[-1],
                'priorpolarity': priorpolarity.split('=')[-1],
                'word': word.split('=')[-1],
                'pos': pos1.split('=')[-1]
            }
    # Delete all conflicting words
#     for w in set(list_to_delete):
#         del lexicon[w]
    return dict(lexicon)


def binary_score(document, lexicon, use_magnitude):
    score = 0
    for sentence in document['content']:
        for word, pos in sentence:
            # TODO: extract lemmas
            lexicon_entry = lexicon.get(word.lower(), None)
            if lexicon_entry is None:
                continue
            if use_magnitude:
                if lexicon_entry['priorpolarity'] == 'positive' and lexicon_entry['type'] == 'strongsubj':
                    score += 2
                elif lexicon_entry['priorpolarity'] == 'positive' and lexicon_entry['type'] == 'weaksubj':
                    score += 1
                elif lexicon_entry['priorpolarity'] == 'negative' and lexicon_entry['type'] == 'strongsubj':
                    score -= 2
                elif lexicon_entry['priorpolarity'] == 'negative' and lexicon_entry['type'] == 'weaksubj':
                    score -= 1
            else:
                if lexicon_entry['priorpolarity'] == 'positive':
                    score += 1
                elif lexicon_entry['priorpolarity'] == 'negative':
                    score -= 1
    return score


def lexicon_classification(document, lexicon, threshold, use_magnitude):
    score = binary_score(document, lexicon, use_magnitude)
    return 1 if score > threshold else 0


def classify_documents_with_lexicon(documents, lexicon, threshold=8, use_magnitude=False):
    predictions = []
    for document in documents:
        predictions.append(lexicon_classification(document, lexicon, threshold, use_magnitude))
    return predictions

In [ ]:
# Read lexicon
lexicon = read_lexicon()

# Extract true labels
labels = [1 if doc['sentiment'] == 'POS' else 0 for doc in reviews]

# Make predictions with a lexicon
predictions = classify_documents_with_lexicon(reviews, lexicon, threshold=8)


In [ ]:
# token_results should be a list of binary indicators; for example [1, 0, 1, ...]
# where 1 indicates a correct classification and 0 an incorrect classification.
token_results = [1 if l == p else 0 for l, p in zip(labels, predictions)]
token_accuracy = sum(token_results)/len(token_results)
print("Accuracy: %0.2f" % token_accuracy)

SyntaxError: ignored

As the sentiment lexicon also has information about the **magnitude** of
sentiment (e.g., *“excellent"* has the same sentiment _polarity_ as *“good"* but it has a higher magnitude), we can take a more fine-grained approach by adding up all
sentiment scores, and deciding the polarity of the movie review using
the sign of the weighted score $S_{weighted}$.

$$S_{weighted}(w_1w_2...w_n) = \sum_{i = 1}^{n}SLex\big[w_i\big]$$


Make sure you define an appropriate threshold for this approach.

#### (Q1.2) Now incorporate magnitude information and report the classification accuracy. Don't forget to use the threshold. (1pt)

In [ ]:
# Find a new threshold
new_threshold = np.ceil(np.mean([binary_score(review, lexicon, True) for review in reviews]))

# Make predictions with a lexicon
predictions = classify_documents_with_lexicon(reviews, lexicon, threshold=new_threshold, use_magnitude=True)

In [ ]:
magnitude_results = [1 if l == p else 0 for l, p in zip(labels, predictions)]
magnitude_accuracy = sum(magnitude_results)/len(magnitude_results)
print("Accuracy: %0.2f" % magnitude_accuracy)

SyntaxError: ignored

#### (Q.1.3) Make a barplot of the two results (0.5pt)

In [ ]:
plt.figure(figsize=(5, 5))
plt.bar([0, 1], [token_accuracy, magnitude_accuracy])
plt.xticks([0, 1], labels=['token', 'magnitude'])
plt.ylabel('Accuracy');

#### (Q1.4) A better threshold (1pt)
Above we have defined a threshold to account for an inherent bias in the dataset: there are more positive than negative words per review.
However, that threshold does not take into account *document length*. Explain why this is a problem and implement an alternative way to compute the threshold.

*Write your answer here.*

In [ ]:
# YOUR CODE HERE

# (2) Naive Bayes (9.5pts)


Your second task is to program a simple Machine Learning approach that operates
on a simple Bag-of-Words (BoW) representation of the text data, as
described by Pang et al. (2002). In this approach, the only features we
will consider are the words in the text themselves, without bringing in
external sources of information. The BoW model is a popular way of
representing texts as vectors, making it
easy to apply classical Machine Learning algorithms on NLP tasks.
However, the BoW representation is also very crude, since it discards
all information related to word order and grammatical structure in the
original text—as the name suggests.

## Writing your own classifier (4pts)

Write your own code to implement the Naive Bayes (NB) classifier. As
a reminder, the Naive Bayes classifier works according to the following
equation:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} P(c|\bar{f}) = \operatorname*{arg\,max}_{c \in C} P(c)\prod^n_{i=1} P(f_i|c)$$
where $C = \{ \text{POS}, \text{NEG} \}$ is the set of possible classes,
$\hat{c} \in C$ is the most probable class, and $\bar{f}$ is the feature
vector. Remember that we use the log of these probabilities when making
a prediction:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} \Big\{\log P(c) + \sum^n_{i=1} \log P(f_i|c)\Big\}$$

You can find more details about Naive Bayes in [Jurafsky &
Martin](https://web.stanford.edu/~jurafsky/slp3/). You can also look at
this helpful
[pseudo-code](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html).

*Note: this section and the next aim to put you in a position to replicate
    Pang et al.'s Naive Bayes results. However, your numerical results
    will differ from theirs, as they used different data.*

**You must write the Naive Bayes training and prediction code from
scratch.** You will not be given credit for using off-the-shelf Machine
Learning libraries.

The data contains the text of the reviews, where each document consists
of the sentences in the review, the sentiment of the review and an index
(cv) that you will later use for cross-validation. The
text has already been tokenised and POS-tagged for you. Your algorithm
should read in the text, **lowercase it**, store the words and their
frequencies in an appropriate data structure that allows for easy
computation of the probabilities used in the Naive Bayes algorithm, and
then make predictions for new instances.


#### (Q2.1) Unseen words (1pt)
The presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive Bayes classifier to equal $0$.
These can be words which are unseen in both positive and negative training reviews (case 1), but also words which are seen in reviews _of only one sentiment class_ in the training dataset (case 2). In both cases, **you should skip these words for both classes at test time**.  What would be the problem instead with skipping words only for one class in case 2?

*Write your answer here.*

#### (Q2.2) Train your classifier on (positive and negative) reviews with cv-value 000-899, and test it on the remaining (positive and negative) reviews cv900–cv999.  Report results using classification accuracy as your evaluation metric. Your  features are the word vocabulary. The value of a feature is the count of that feature (word) in the document. (2pts)


In [64]:
class NaiveBayesClassifier:
    def __init__(self, k=None):
        self.k = k

    def _skip_word(self, word):
        """
        Determines whether we should skip the words unseen by one of the classes.

        Args:
            word (str): a word to check

        Returns:
            (bool): whether we should skip the word
        """
        for c in self.class_priors:
            if self.class_condtitional_probs[word][c] == 0:
                return True
        return False

    def train(self, data):
        """
        Trains the NB classifier.

        Args:
            data (list[dict]): training data with dicts of format
                {'cv': int, 'sentiment': str, 'content': list}
        """
        self.vocab = self.extract_vocab(data)
        self.n_docs = len(data)
        self.class_priors = self.compute_class_priors(data)
        self.class_condtitional_probs = self.compute_class_condtitional_probs(data)

    def predict(self, data):
        """
        Predicts the labels of the documents.

        Args:
            data (list[dict]): test data with dicts of format
                {'cv': int, 'sentiment': str, 'content': list}
        """
        predictions = []
        for document in data:
            class_posteriors = {}
            for c in self.class_priors:
                class_posteriors[c] = np.log(self.class_priors[c])
                words = self.extract_words(document)
                for word in words:
                    class_posteriors[c] += np.log(self.class_condtitional_probs[word][c])
            predictions.append(max(class_posteriors, key=class_posteriors.get))
        return predictions

    def extract_words(self, document, training=False): #done
        """
        Exctracts the words into a list for a document.

        Args:
            document (dict): {'cv': int, 'sentiment': str, 'content': list}
            training (bool): Indicates whether the method is used in the training
                or testing context. Influences whether we keep unseen words.
                Default value: False.

        Returns:
            words (list[str]): a list of words
        """
        words = []
        for word in document['content']:

            # Skip unseen words
            if word not in self.class_condtitional_probs:
                continue

            # Skip words unseen by any class
            if not training and self._skip_word(word):
                continue

            words.append(word)
        return words

    def extract_vocab(self, documents): # done
        """
        Extracts a vocabulary from the provided data.

        Args:
            documents (list[dict]): {'cv': int, 'sentiment': str, 'content': list}

        Returns:
            vocab (Counter): all the words present in the data with corresponding counts
        """
        vocab = Counter()
        for document in documents:
            for word in document['content']:
                vocab[word] += 1
        return vocab

    def compute_class_counts(self, documents):
        """
        Counts the number of appearences of each class in the data.

        Args:
            documents (list[dict]): {'cv': int, 'sentiment': str, 'content': list}

        Returns:
            (Counter): counts of each class
        """
        return Counter([d['sentiment'] for d in documents])

    def compute_class_conditional_counts(self, documents):
        """
        Counts the number of appearences of each word in each class in the data.

        Args:
            documents (list[dict]): {'cv': int, 'sentiment': str, 'content': list}

        Returns:
            (dict): {'<class>': {<word>: int}}
        """
        class_conditional_counts = defaultdict(Counter)

        for document in documents:
            sentiment = document['sentiment']
            for word in document['content']:
                class_conditional_counts[sentiment][word] += 1
        return dict(class_conditional_counts)

    def compute_class_priors(self, documents):
        """
        Computes the class priors based on the class counts.

        Args:
            documents (list[dict]): {'cv': int, 'sentiment': str, 'content': list}

        Returns:
            class_priors (dict): {'<class>': float}
        """
        class_counts = self.compute_class_counts(documents)
        class_priors = {}
        for c, count in class_counts.items():
            class_priors[c] = count / self.n_docs
        return class_priors

    def compute_class_condtitional_probs(self, documents):
        """
        Computes the class conditional likelihood based on the word counts for each class.

        Args:
            documents (list[dict]): {'cv': int, 'sentiment': str, 'content': list}
            k (Optional[float]): smoothing factor

        Returns:
            class_condtitional_probs (dict): {'<word>': {<class>: float}}
        """
        class_conditional_counts = self.compute_class_conditional_counts(documents)
        class_condtitional_probs = defaultdict(dict)

        for c, counts in class_conditional_counts.items():
            counts_vector = np.array([counts[word] for word in self.vocab])
            if self.k:
                probs_vector = counts_vector + self.k / (counts_vector.sum() + len(self.vocab)*self.k)
            else:
                probs_vector = counts_vector / counts_vector.sum()

            for i, word in enumerate(self.vocab):
                class_condtitional_probs[word][c] = probs_vector[i]

        return dict(class_condtitional_probs)


class DataLoader:
    def __init__(self, data):
        self.data = data
        self.stemmer = PorterStemmer()

    def get_data(self, start_index, end_index, sentiment=None, ngrams_n=None, use_stemming=False, POS = False, pos_to_keep = None):
        new_documents = []

        if sentiment == None:
            subset = [document for document in self.data if (start_index <= document['cv'] <= end_index)]
        else:
            subset = [document for document in self.data if (start_index <= document['cv'] <= end_index) and (document['sentiment'] == sentiment)]

        # return the data in a cleaner format
        for document in subset:
            new_document = {}

            words, pos_tags = self._flatten_content(document, use_stemming)
            #print(type(words))


            if ngrams_n is not None:
                for n in range(2, ngrams_n+1):
                    words += list(ngrams(words, n))
                    pos_tags += list(ngrams(pos_tags, n))

            # essentially used to only keep the POS tags in pos_to_keep, and if this is a 2d list we want to replace all the POS tags in the twod list with the first element of this 2d list -> all verbs are just VB
            if POS:
                if pos_to_keep is not None:
                    if isinstance(pos_to_keep[0], list):
                      temp_words = []
                      for pos_type in pos_to_keep:
                          #print(pos_type[0])
                          temp_words += [word + '_' + pos_type[0] for word, pos in zip(words,pos_tags) if pos in pos_type]
                      words = temp_words
                    else:
                        words = [word + '_' + pos.strip() for word, pos in zip(words,pos_tags) if pos in pos_to_keep]
                else:
                  words = [word + '_' + pos.strip() for word, pos in zip(words,pos_tags)]

            new_document['content'] = words
            #new_document['pos'] = pos_tags -> instead adding it behind the data like word_POS seems much easier.
            new_document['cv'] = document['cv']
            new_document['sentiment'] = document['sentiment']
            new_documents.append(new_document)

        return new_documents

    def _flatten_content(self, document, use_stemming):
        words, pos_tags = [], []
        for sentence in document['content']:
            for word, pos in sentence:
                words.append(self._clean(word, use_stemming))
                pos_tags.append(pos)
        return words, pos_tags


    def _clean(self, word, use_stemming):
        """
        Preprocesses the word by lowercasing it and applying stemming (optional).

        Args:
            word (str): the word to clean

        Returns:
            clean_word (str): the cleaned word
        """
        clean_word = word.lower()
        if use_stemming:
            clean_word = self._stem(clean_word)
        return clean_word

    def _stem(self, word):
        """
        Stems the word.

        Args:
            word (str): the word to stem

        Returns:
            (str): the stemmed word
        """
        return self.stemmer.stem(word)


def evaluate_model(model, test_data):
    # Predict the classes on the test data
    predictions = model.predict(test_data)

    # Extract the true labels and compare them to the predictions
    labels = [d['sentiment'] for d in test_data]
    correct_predictions = [1 if p == l else 0 for p, l in zip(predictions, labels)]

    # Compute accuracy
    accuracy = sum(correct_predictions) / len(correct_predictions)
    return accuracy


data_loader = DataLoader(reviews)

In [6]:
# Split the data
train, test = data_loader.get_data(0, 899), data_loader.get_data(900, 999)

# Train the NB classifier
nb = NaiveBayesClassifier()
nb.train(train)

# Compute the accuracy
evaluate_model(nb, test)

0.825

#### (Q2.3) Would you consider accuracy to also be a good way to evaluate your classifier in a situation where 90% of your data instances are of positive movie reviews? (1pt)

Simulate this scenario by keeping the positive reviews
data unchanged, but only using negative reviews cv000–cv089 for
training, and cv900–cv909 for testing. Calculate the classification
accuracy, and explain what changed.

*Write your answer here.*

In [7]:
train_negative = data_loader.get_data(0,89, 'NEG')
test_negative = data_loader.get_data(900, 909, 'NEG')

train_positive = data_loader.get_data(0, 899, 'POS')
test_positive = data_loader.get_data(900, 999, 'POS')

train_total = train_negative + train_positive
test_total = test_negative + test_positive
print("Number of documents in training set:", len(train_total))
print("Number of documents in test set:", len(test_total))

# Train the NB classifier
nb = NaiveBayesClassifier()
nb.train(train_total)

evaluate_model(nb, test_total)

Number of documents in training set: 990
Number of documents in test set: 110


0.6

## Smoothing (1pt)

As mentioned above, the presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive
Bayes classifier to be $0$, thus making that particular test instance
undecidable. The standard way to mitigate this effect (as well as to
give more clout to rare words) is to use smoothing, in which the
probability fraction
$$\frac{\text{count}(w_i, c)}{\sum\limits_{w\in V} \text{count}(w, c)}$$ for a word
$w_i$ becomes
$$\frac{\text{count}(w_i, c) + \text{smoothing}(w_i)}{\sum\limits_{w\in V} \text{count}(w, c) + \sum\limits_{w \in V} \text{smoothing}(w)}$$





#### (Q2.4) Implement Laplace feature smoothing (1pt)
Implement Laplace smoothing, i.e., smoothing with a constant value ($smoothing(w) = \kappa, \forall w \in V$), in your Naive
Bayes classifier’s code, and report the accuracy.
Use $\kappa = 1$.

In [8]:
# Train the NB classifier
nb = NaiveBayesClassifier(k=1)
nb.train(train_total)

evaluate_model(nb, test_total)

0.9090909090909091

## Cross-Validation (1.5pts)

A serious danger in using Machine Learning on small datasets, with many
iterations of slightly different versions of the algorithms, is ending up with Type III errors, also called the “testing hypotheses
suggested by the data” errors. This type of error occurs when we make
repeated improvements to our classifiers by playing with features and
their processing, but we don’t get a fresh, never-before seen test
dataset every time. Thus, we risk developing a classifier that gets better
and better on our data, but only gets worse at generalizing to new, unseen data. In other words, we risk developping a classifier that overfits.

A simple method to guard against Type III errors is to use
Cross-Validation. In **N-fold Cross-Validation**, we divide the data into N
distinct chunks, or folds. Then, we repeat the experiment N times: each
time holding out one of the folds for testing, training our classifier
on the remaining N - 1 data folds, and reporting performance on the
held-out fold. We can use different strategies for dividing the data:

-   Consecutive splitting:
  - cv000–cv099 = Split 1
  - cv100–cv199 = Split 2
  - etc.
  
-   Round-robin splitting (mod 10):
  - cv000, cv010, cv020, … = Split 1
  - cv001, cv011, cv021, … = Split 2
  - etc.

-   Random sampling/splitting
  - Not used here (but you may choose to split this way in a non-educational situation)



#### (Q2.5) Write the code to implement 10-fold cross-validation using round-robin splitting for your Naive Bayes classifier from Q2.4 and compute the 10 accuracies. Report the final performance, which is the average of the performances per fold. If all splits perform equally well, this is a good sign. (1pt)

In [9]:
def cross_validation(data_loader, n_splits=10, total_points=1000, use_stemming=False, ngrams_n=None, k=None, verbose=False):
    split_size = total_points // n_splits

    results = []
    for i in range(0, total_points, split_size):
        if verbose:
            print(f"Split {i}:{i+split_size}", end=" | ")
        test_data = data_loader.get_data(i, i+split_size, use_stemming=use_stemming, ngrams_n=ngrams_n)

        train_start_2, train_end_2 = i+split_size, total_points
        train_data = []
        if i != 0:
            train_data += data_loader.get_data(0, 0+i, use_stemming=use_stemming, ngrams_n=ngrams_n)
        if i+split_size != total_points:
            train_data += data_loader.get_data(i+split_size, total_points, use_stemming=use_stemming, ngrams_n=ngrams_n)

        model = NaiveBayesClassifier(k=k)
        model.train(train_data)

        accuracy = evaluate_model(model, test_data)
        results.append(accuracy)
        if verbose:
            print(f"accuracy: {accuracy}")

    results = np.array(results)
    print(f"\nAverage accuracy: {np.mean(results)}")
    print(f"Max accuracy: {np.max(results)}")
    print(f"Min accuracy: {np.min(results)}")
    print(f"Variance: {((results - results.mean())**2).mean()}")

    return results


In [10]:
results = cross_validation(data_loader, use_stemming=False, k=None, ngrams_n=None, verbose=True)

Split 0:100 | accuracy: 0.7772277227722773
Split 100:200 | accuracy: 0.8267326732673267
Split 200:300 | accuracy: 0.8465346534653465
Split 300:400 | accuracy: 0.8217821782178217
Split 400:500 | accuracy: 0.8168316831683168
Split 500:600 | accuracy: 0.8267326732673267
Split 600:700 | accuracy: 0.8267326732673267
Split 700:800 | accuracy: 0.8316831683168316
Split 800:900 | accuracy: 0.7821782178217822
Split 900:1000 | accuracy: 0.825

Average accuracy: 0.8181435643564356
Max accuracy: 0.8465346534653465
Min accuracy: 0.7772277227722773
Variance: 0.0004240276688559936


In [11]:
results = cross_validation(data_loader, use_stemming=False, k=1, ngrams_n=None, verbose=True)

Split 0:100 | accuracy: 0.6485148514851485
Split 100:200 | accuracy: 0.693069306930693
Split 200:300 | 

KeyboardInterrupt: ignored

#### (Q2.6) Report the variance of the 10 accuracy scores. (0.5pt)

**Please report all future results using 10-fold cross-validation now
(unless told to use the held-out test set).** Note: you're not allowed to use a library for computing the variance.

In [ ]:
((results - results.mean())**2).mean()

## Features, overfitting, and the curse of dimensionality

In the Bag-of-Words model, ideally we would like each distinct word in
the text to be mapped to its own dimension in the output vector
representation. However, real world text is messy, and we need to decide
on what we consider to be a word. For example, is “`word`" different
from “`Word`", from “`word`”, or from “`words`"? Too strict a
definition, and the number of features explodes, while our algorithm
fails to learn anything generalisable. Too lax, and we risk destroying
our learning signal. In the following section, you will learn about
confronting the feature sparsity and the overfitting problems as they
occur in NLP classification tasks.

### Stemming (1.5pts)

To make your algorithm more robust, use stemming and hash different inflections of a word to the same feature in the BoW vector space. Please use the [Porter stemming
    algorithm](http://www.nltk.org/howto/stem.html) from NLTK.



In [ ]:
# instead of implementing it here it is implemented in the large code block above

#### (Q2.7): How does the performance of your classifier change when you use stemming on your training and test datasets? (1pt)
Use cross-validation to evaluate the classifier.


In [ ]:
results = cross_validation(data_loader, use_stemming=True, k=1, ngrams_n=None, verbose=False)

#### (Q2.8) What happens to the number of features (i.e., the size of the vocabulary) when using stemming as opposed to (Q2.4)? (0.5pt)
Give actual numbers. You can use the held-out training set to determine these.

In [ ]:
nb = NaiveBayesClassifier()
no_stem_vocab = nb.extract_vocab(data_loader.get_data(0, 899, use_stemming=False))
stem_vocab = nb.extract_vocab(data_loader.get_data(0, 899, use_stemming=True))

print(f"Vocab size without stemming: {len(no_stem_vocab)}")
print(f"Vocab size with stemming: {len(stem_vocab)}")

### N-grams (1.5pts)

A simple way of retaining some of the word
order information when using bag-of-words representations is to use **n-gram** features.






#### (Q2.9) Retrain your classifier from (Q2.4) using **unigrams+bigrams** and **unigrams+bigrams+trigrams** as features. (1pt)
Report accuracy and compare it with that of the approaches you have previously implemented. You are allowed to use NLTK to build n-grams from sentences.

In [ ]:
results_n2 = cross_validation(data_loader, use_stemming=False, k=1, ngrams_n=2, verbose=True)

In [ ]:
results_n3 = cross_validation(data_loader, use_stemming=False, k=1, ngrams_n=3, verbose=True)


#### Q2.10: How many features does the BoW model have to take into account now? (0.5pt)
How would you expect the number of features to increase theoretically (e.g., linear, square, cubed, exponential)? How does this number compare, in practice, to the number of features at (Q2.8)?

Use the held-out training set once again for this.


*Write your answer here.*

In [ ]:
nb = NaiveBayesClassifier()
vocab_n2 = nb.extract_vocab(data_loader.get_data(0, 899, use_stemming=False, ngrams_n=2))
vocab_n3 = nb.extract_vocab(data_loader.get_data(0, 899, use_stemming=False, ngrams_n=3))

print(f"Vocab size unigrams+bigrams: {len(vocab_n2)}")
print(f"Vocab size unigrams+bigrams+trigrams: {len(vocab_n3)}")

# (3) Support Vector Machines (4pts)

Though simple to understand, implement, and debug, one
major problem with the Naive Bayes classifier is that its performance
deteriorates (becomes skewed) when it is being used with features which
are not independent (i.e., are correlated). Another popular classifier
that doesn’t scale as well to big data, and is not as simple to debug as
Naive Bayes, but that doesn’t assume feature independence is the Support
Vector Machine (SVM) classifier.

You can find more details about SVMs in Chapter 7 of Bishop: Pattern Recognition and Machine Learning.
Other sources for learning SVM:
* http://web.mit.edu/zoya/www/SVM.pdf
* http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf
* https://pythonprogramming.net/support-vector-machine-intro-machine-learning-tutorial/







Use the scikit-learn implementation of
[SVM](http://scikit-learn.org/stable/modules/svm.html) with the default parameters. (You are not expected to perform any hyperparameter tuning, but feel free to do it if you think it gives you good insights for the discussion in question 5.)



#### (Q3.1): Train SVM and compare to Naive Bayes (2pts)

Train an SVM classifier (sklearn.svm.LinearSVC) using the features collected for Naive Bayes. Compare the
classification performance of the SVM classifier to that of the Naive
Bayes classifier with smoothing.
Use cross-validation to evaluate the performance of the classifiers.



In [51]:
def encode_data(data, word2id):
    vocab_size = len(word2id)
    encoded_data = []

    for document in data:
        encoded_document = np.zeros(vocab_size)
        for word in document['content']:
            if word in word2id:
                encoded_document[word2id[word]] += 1
        encoded_data.append(encoded_document)
    return np.array(encoded_data)

def extract_labels(data, label2id):
    labels = []
    for document in data:
        labels.append(label2id[document['sentiment']])
    return np.array(labels)


def svm_cross_validation(data_loader, n_splits=10, total_points=1000, use_stemming=False, ngrams_n=None, verbose=False, POS = False, pos_to_keep = None):
    split_size = total_points // n_splits

    results = []
    for i in range(0, total_points, split_size):
        if verbose:
            print(f"Split {i}:{i+split_size}", end=" | ")
        test_data = data_loader.get_data(i, i+split_size, use_stemming=use_stemming, ngrams_n=ngrams_n, POS = POS, pos_to_keep = pos_to_keep)

        train_data = []
        if i != 0:
            train_data += data_loader.get_data(0, 0+i, use_stemming=use_stemming, ngrams_n=ngrams_n, POS = POS, pos_to_keep = pos_to_keep)
        if i+split_size != total_points:
            train_data += data_loader.get_data(i+split_size, total_points, use_stemming=use_stemming, ngrams_n=ngrams_n, POS = POS, pos_to_keep = pos_to_keep)

        nb = NaiveBayesClassifier()
        vocab = nb.extract_vocab(train_data)
        id2word = dict(enumerate(vocab))
        id2label = dict(enumerate(set(d['sentiment'] for d in train_data)))
        word2id = {v: k for k, v in id2word.items()}
        label2id = {v: k for k, v in id2label.items()}

        X_train, y_train = encode_data(train_data, word2id), extract_labels(train_data, label2id)
        X_test, y_test = encode_data(test_data, word2id), extract_labels(test_data, label2id)

        #model = LinearSVC(dual='auto')
        model = LinearSVC()
        model.fit(X_train, y_train)

        accuracy = model.score(X_test, y_test)
        results.append(accuracy)
        if verbose:
            print(f"accuracy: {accuracy}")

    results = np.array(results)
    print(f"\nAverage accuracy: {np.mean(results)}")
    print(f"Max accuracy: {np.max(results)}")
    print(f"Min accuracy: {np.min(results)}")
    print(f"Variance: {((results - results.mean())**2).mean()}")

    return results


In [37]:
# Prep the data
nb = NaiveBayesClassifier()

train_data, test_data = data_loader.get_data(0, 899), data_loader.get_data(900, 999)
vocab = nb.extract_vocab(train_data)
id2word = dict(enumerate(vocab))
id2label = dict(enumerate(set(d['sentiment'] for d in train_data)))
word2id = {v: k for k, v in id2word.items()}
label2id = {v: k for k, v in id2label.items()}

X_train, y_train = encode_data(train_data, word2id), extract_labels(train_data, label2id)
X_test, y_test = encode_data(test_data, word2id), extract_labels(test_data, label2id)

In [31]:
#svm = LinearSVC(dual='auto') #this doesnt run in colab for some reason #TODO
svm = LinearSVC()
svm.fit(X_train, y_train)

svm.score(X_test, y_test)

0.88

In [28]:
# Compare score with accuracy to confirm
matches = (svm.predict(X_test) == y_test).astype(int)
matches.sum() / len(matches)

0.88

In [29]:
svm_cross_validation(data_loader, verbose=True)

Split 0:100 | accuracy: 0.8316831683168316
Split 100:200 | 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


accuracy: 0.8514851485148515
Split 200:300 | 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


accuracy: 0.8465346534653465
Split 300:400 | 

...

accuracy: 0.806930693069307
Split 800:900 | 

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


accuracy: 0.8168316831683168
Split 900:1000 | accuracy: 0.88

Average accuracy: 0.8444356435643565
Max accuracy: 0.88
Min accuracy: 0.806930693069307
Variance: 0.00048745181844917174


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([0.83168317, 0.85148515, 0.84653465, 0.85643564, 0.84158416,
       0.87623762, 0.83663366, 0.80693069, 0.81683168, 0.88      ])

### POS disambiguation (2pts)

Now add in part-of-speech features. You will find the
movie review dataset has already been POS-tagged for you ([here](https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf) you find the tagset). Try to
replicate the results obtained by Pang et al. (2002).



####(Q3.2) Replace your features with word+POS features, and report performance with the SVM. Use cross-validation to evaluate the classifier and compare the results with (Q3.1). Does part-of-speech information help? Explain why this may be the case. (1pt)


In [38]:
# YOUR CODE HERE
# Prep the data to include POS tags
nb = NaiveBayesClassifier()

train_data, test_data = data_loader.get_data(0, 899, POS = True), data_loader.get_data(900, 999, POS = True)
vocab = nb.extract_vocab(train_data)
id2word = dict(enumerate(vocab))
id2label = dict(enumerate(set(d['sentiment'] for d in train_data)))
word2id = {v: k for k, v in id2word.items()}
label2id = {v: k for k, v in id2label.items()}

X_train, y_train = encode_data(train_data, word2id), extract_labels(train_data, label2id)
X_test, y_test = encode_data(test_data, word2id), extract_labels(test_data, label2id)

In [39]:
svm = LinearSVC()
svm.fit(X_train, y_train)

svm.score(X_test, y_test)

0.85

In [41]:
svm_cross_validation(data_loader, verbose=False, POS = True)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w


Average accuracy: 0.8473811881188119
Max accuracy: 0.8811881188118812
Min accuracy: 0.8316831683168316
Variance: 0.00020250879815704377


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([0.84158416, 0.84653465, 0.83168317, 0.83663366, 0.84653465,
       0.88118812, 0.86138614, 0.83663366, 0.83663366, 0.855     ])

*Write your answer here.*

#### (Q3.3) Discard all closed-class words from your data (keep only nouns, verbs, adjectives, and adverbs), and report performance. Does this help? Use cross-validation to evaluate the classifier and compare the results with (Q3.2). Are closed-class words detrimental to the classifier? Explain why this may be the case. (1pt)

In [61]:
# YOUR CODE HERE
# this is the list of all POS tags in our document
#['JJ', '$', '-RRB-', 'VBN', 'RBS', 'WRB', 'PRP$', 'TO', 'WP$', 'FW', 'NNS', 'VBZ', 'RB', 'IN', 'CC', 'JJR', '.', 'VB', 'RBR', 'MD', 'WDT', 'VBP', 'WP', ':', 'EX', 'VBG', '-LRB-', ',', 'RP', '``', 'UH', 'POS', "''", 'NNP'
#, 'DT', 'LS', 'PDT', 'NNPS', 'SYM', 'VBD', 'NN', 'JJS', 'CD', '#', 'PRP']

#Want to keep nouns (NN, NNS), verbs (VB, VBD VBG VBN VBP VBZ), ajectives (JJR, JJS), adverbs (RB, RBR, RBS)

pos_to_keep = ['NN', 'NNS','VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
pos_to_keep_merge = [['NN', 'NNS'], ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'], ['JJ', 'JJR', 'JJS'], ['RB', 'RBR', 'RBS']]

nb = NaiveBayesClassifier()

train_data, test_data = data_loader.get_data(0, 899, POS = True, pos_to_keep = pos_to_keep), data_loader.get_data(900, 999, POS = True, pos_to_keep = pos_to_keep)
vocab = nb.extract_vocab(train_data)
id2word = dict(enumerate(vocab))
id2label = dict(enumerate(set(d['sentiment'] for d in train_data)))
word2id = {v: k for k, v in id2word.items()}
label2id = {v: k for k, v in id2label.items()}

X_train, y_train = encode_data(train_data, word2id), extract_labels(train_data, label2id)
X_test, y_test = encode_data(test_data, word2id), extract_labels(test_data, label2id)


In [62]:
svm = LinearSVC()
svm.fit(X_train, y_train)

svm.score(X_test, y_test)

0.875

In [57]:
svm_cross_validation(data_loader, verbose=False, POS = True, pos_to_keep = pos_to_keep)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



Average accuracy: 0.8488910891089109
Max accuracy: 0.88
Min accuracy: 0.8217821782178217
Variance: 0.00042993784923046836


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([0.82673267, 0.84653465, 0.86633663, 0.86633663, 0.86138614,
       0.83168317, 0.82178218, 0.86633663, 0.82178218, 0.88      ])

In [65]:
svm_cross_validation(data_loader, verbose=False, POS = True, pos_to_keep = pos_to_keep_merge)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



Average accuracy: 0.8518712871287129
Max accuracy: 0.89
Min accuracy: 0.8267326732673267
Variance: 0.0004055179884325073


array([0.82673267, 0.83168317, 0.87623762, 0.87128713, 0.85643564,
       0.84158416, 0.83168317, 0.84653465, 0.84653465, 0.89      ])

*Write your answer here.*

# (4) Discussion (max. 500 words). (5pts)

> Based on your experiments, what are the effective features and techniques in sentiment analysis? What information do different features encode?
Why is this important? What are the limitations of these features and techniques?



*Write your answer here in up to 500 words (-0.25pt for >50 extra words, -0.5 points for >100 extra words, ...)*.


# Submission


In [ ]:
# Write your names and student numbers here:
# Student 1 #12345
# Student 2 #12345

**That's it!**

- Check if you answered all questions fully and correctly.
- Download your completed notebook using `File -> Download .ipynb`
- Check if your answers are all included in the file you submit.
- Submit your .ipynb file via *Canvas*. One submission per group.